In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw9.ipynb")

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
pd.set_option("future.no_silent_downcasting", True)
pd.options.mode.chained_assignment = None

## Name: [Your name] and (your onyen)

I encourage discussing ideas and brainstorming with your peers, but the final text, code, and comments in this homework assignment MUST be 100% written by you as mentioned in syllabus.



# Homework 9: Decision Trees and Over/Under Fitting (25 points)

Please complete this notebook by filling in the cells provided.

!! Please submit a zip and a PDF of your notebook. You can make a PDF from JupyterHub by 1. File->Print or 2. Command+P (on Mac), then choose 'Save to PDF'. This saves the file on your laptop. Most problems will be manually graded, but there a couple autograded questions.  No private autograded, so if you pass in JupyterHub, you'll get full Autograder credit on Gradescope. Please make sure all your code is visible and not cut off in the pdf.

!! Points will also be deducted if you do not include your name and Onyen in the Markdown cell above!!

## Problem 1. You need a loan?  Hmm, let me decide (25 points)
Many of you have expressed interest in financial technology, so for this problem you will be using historic data from a bank about whether or not people were granted loans.  You'll be using this data to create a predictor for whether the bank would grant loans to future applicants.


### **Problem 1.1 Data Preparation (4 points)**
First, load the loan data into a table named `loan_data_raw`.


In [ ]:
loan_data_raw = pd.read_csv('loan_data.csv')

#### Problem 1.1.1 (1 point)
We don't need the `Loan_ID` column, so drop that column and store the resulting table in `loan_data`.

In [ ]:
# drop Loan_ID column
loan_data= ...
loan_data

In [ ]:
grader.check("q111")

#### Problem 1.1.2 (1 point)
Some of the values are missing from the table, so write the code to drop rows from `loan_data` that have any missing values, and store the result in `loan_data_clean`.

In [ ]:
# remove rows with missing values
loan_data_clean= ...
loan_data_clean

#### Problem 1.1.3 (2 points)
You'll be using functions from `scikit-learn`, which don't know how to handle text values like `Male` or `Female` or `Yes` or `No`, so you'll have to turn them into numbers. For the `Gender` and `Married` columns, use boolean indexing to do this by assigning 0 for `Male` and 1 for `Female` and 0 for `No` and 1 for `Yes` (just like we did in lecture demo).  Some of the code is already provided, you fill in the rest.  

**WARNING:** running this cell twice will set `Gender` and `Married` to all zeroes!  (think about why this is so)  If you do this accidentally, you can always just run the cell above that drops values to re-create `loan_data_clean`.

In [ ]:
is_Female = ...
loan_data_clean['Gender'] = is_Female.astype(int)
is_Yes = (loan_data_clean['Married']=='Yes')
loan_data_clean['Married'] = ...
loan_data_clean

#### Problem 1.1.4 (0 points)
Copy the code below that will do the same thing (a faster way) for the remaining columns.
```Python
loan_data_clean['Loan_Status'] = loan_data_clean.Loan_Status.replace({'N': 0, 'Y': 1})
loan_data_clean['Property_Area'] = loan_data_clean.Property_Area.replace({'Rural':0, 'Urban':1, 'Semiurban':2})
loan_data_clean['Education'] = loan_data_clean.Education.replace({'Graduate':0, 'Not Graduate':1})
loan_data_clean['Dependents'] = loan_data_clean.Dependents.replace({'0':0, '1':1,'2':2,'3+':3})
loan_data_clean = loan_data_clean.astype(float)
loan_data_clean
```


In [ ]:
# Copy code here and run.
...

### **Problem 1.2 Train and Test, part 1 (12 points)**
Next, you'll be creating your training set and your test set.  


#### Problem 1.2.1 (2 points)
As we did in class and lab, create the training data from 80% of `loan_data_clean` by using random sampling.  The create the test set by dropping from `loan_data_clean` all the rows that are in the training set.  We set the random seed below to make the sampling reproducible so that everyone will be working with the same training and test sets.

In [ ]:
np.random.seed(420) # this makes the sampling reproducible, so everyone has same train and test
train_loan = ...
test_loan = ...
print(train_loan.shape, test_loan.shape)

#### Problem 1.2.2 (2 points)
You will be trying to predict `Loan_Status` from all the other data.  Create the approriate `X_train_loan`, `y_train_loan`, `X_test_loan`, and `y_test_loan` tables for the train and test sets.

In [ ]:
X_train_loan = ...
y_train_loan = ...
X_test_loan = ...
y_test_loan = ...

#### Problem 1.2.3 (2 points)
Finally, use the `DecisionTreeClassifier` to create a decision tree to use for prediction.  Finish the code to print the scores of the classifier on the train and test sets.

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

T = DecisionTreeClassifier(max_depth=3)
T.fit(X_train_loan, y_train_loan)
train_score = ...
test_score = ...
print('Score on train:', train_score)
print('Score on test:', test_score)

fig, ax = plt.subplots(1, figsize = (20, 20))
p = plot_tree(T, filled = True, feature_names = X_train_loan.columns)

#### Problem 1.2.4 (2 points)
Looking at the decision tree, which feature is the most important for predicting if a loan would be approved? 

*Write your answer here*

#### Problem 1.2.5 (2 points)
Next, we're going to gather some data for creating decision trees of different maximum tree depths.  Finish the code below to call the decision tree classifier with different maximum depths.

In [ ]:
train_scores=[]
test_scores=[]
r=range(1,13)
for i in r:
    T = ...
    T.fit(X_train_loan, y_train_loan)
    train_scores.append(T.score(X_train_loan, y_train_loan))
    test_scores.append(T.score(X_test_loan, y_test_loan))
fig, ax = plt.subplots(1)
sns.lineplot(x=r,y=train_scores,label='train')
sns.lineplot(x=r,y=test_scores,label='test')
ax.set_ylabel('Prediction score', fontsize=14)
ax.set_xlabel('Max tree depth', fontsize=14)
ax.legend()


#### Problem 1.2.6 (2 points)
Observe the generated plot.
1. Does the accuracy improve for the **train** set as the maximum tree depth increases?
2. Does the accuracy improve for the **test** set as the maximum tree depth increases?

*Write your answer here*

### **Problem 1.3 Train and Test, part 2 (9 points)**
You may have noticed that the `Credit_History` feature dominates the loan decision process (which makes sense!)  Using just this feature (max tree depth == 1), the prediction score is over 80%.  But what if you didn't have access to that feature?  How well could you predict what the bank would do?

#### Problem 1.3.1 (1 point)
Finish the code below to drop the `Credit_History` column from your train and test data.

In [ ]:
X2_train_loan = ...
X2_test_loan = ...

#### Problem 1.3.2 (1 point)
Finish the code below to create a new classifier fit to the new train and test data.

In [ ]:
T2 = DecisionTreeClassifier(max_depth=3)
# write your code here
...

print('Score on train:', T2.score(X2_train_loan, y_train_loan))
print('Score on test:', T2.score(X2_test_loan, y_test_loan))

fig, ax = plt.subplots(1, figsize = (20, 20))
p = plot_tree(T2, filled = True, feature_names = X2_train_loan.columns)

#### Problem 1.3.3 (1 point)
How do the new scores for the train and test data compare to the previous scores that you printed?  Explain why you might have expected this.

*Write your answer here*

#### Problem 1.3.4 (1 point)
Once again, finish the code to gather prediction scores at different max tree depths.

In [ ]:
train_scores=[]
test_scores=[]
r=range(1,13)
for i in r:
    T = DecisionTreeClassifier(max_depth=i)
    # write your code here
    ...
    train_scores.append(T.score(X2_train_loan, y_train_loan))
    test_scores.append(T.score(X2_test_loan, y_test_loan))
fig, ax = plt.subplots(1)
sns.lineplot(x=r,y=train_scores,label='train')
sns.lineplot(x=r,y=test_scores,label='test')
ax.set_ylabel('Prediction score', fontsize=14)
ax.set_xlabel('Max tree depth', fontsize=14)
ax.legend()

#### Probelm 1.3.5 (2 points)

You should notice that increasing the tree depth dramatically improves the prediction score for the train set. This is because without `Credit_History`, there is no one primary feature correlated to the outcome, so we need more features to explain the loan decision data. At roughly what tree depth is the predictor able to predict as well for the train data as the previous predictor did using just `Credit_History`? (i.e. > 80%)?

*Write your answer here!*

#### Problem 1.3.6 (3 points)
You should see that for the test data, the predictor never gets near to the 80% mark.  No matter how deep the tree, the prediction score for the test data does not improve the same way as it did for the train data.  Why is this?

*Write your answer here!*

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Submit zip file and PDF to Gradescope Homework9

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)